Public Score: 0.74641

In [112]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from datetime import datetime

In [113]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

df = pd.concat([train, test], sort=False)

In [114]:
df.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [115]:
df["Age"] = df["Age"].fillna(df["Age"].median())
df["Embarked"] = df["Embarked"].fillna("S")

sex = pd.get_dummies(df["Sex"], drop_first=True)
ticket = pd.get_dummies(df["Ticket"], drop_first=True)
embark = pd.get_dummies(df["Embarked"], drop_first=True)

df = pd.concat([df, sex], axis=1)
df = pd.concat([df, ticket], axis=1)
df = pd.concat([df, embark], axis=1)
df = df.drop(["Sex", "Ticket", "Embarked", "Cabin", "Name"], axis=1)

df.head(2)

train = df[:len(train)]
test = df[len(train):]


In [116]:
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
              ..
W.E.P. 5734    0
W/C 14208      0
WE/P 5735      0
Q              0
S              0
Length: 938, dtype: int64

In [117]:
X = train.loc[:, 'Pclass':'S']
y = train["Survived"]

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33, 
                                                    random_state=0, 
                                                    shuffle =True)

In [119]:
XGBModel = XGBClassifier(random_state=0)

# Train the model
XGBModel.fit(X_train, y_train)


XGBModel.score(X_test, y_test)

0.8101694915254237

In [120]:
pred = XGBModel.predict(test.loc[:, 'Pclass':'S'])

output = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': pred})

# CSVファイルとして出力
output.to_csv(f"./output/{datetime.now().strftime('%Y%m%d-%H%M')}-submission.csv", index=False)